In [29]:
using Plots, Iterators, Rsvg
plotlyjs()

Plots.PlotlyJSBackend()

# ActiveDendrite.m

In [1]:
function ActiveDendrite()
    Total_Equations = 8;  #Solve for this number of interacting Neurons
    DT = 0.04;  #Time increment
    Final_Time = 30;   #Final time value for calculation
    Last = Int64(Final_Time/DT + 1);  #Last time step
    Time = DT*(0:Last-1);  #Time vector
    Tau = 0.97;  #Neural time constants in msec
    TauR = 5.6
    WTS = [1 2 2 1];  #Runge-Kutta Coefficient weights


    X = Array{Float64}(Total_Equations, Last)
    K = Array{Float64}(Total_Equations, 4)
    Weights = Array{Float64}(Total_Equations, 4)


    for NU = 1:Total_Equations;  #Initialize
        X[NU, :] = zeros(1, Last);  #Vector to store response of Neuron #1
        K[NU, :] = zeros(1, 4);  #Runge-Kutta terms	
        Weights[NU, :] = WTS;  #Make into matrix for efficiency in main loop
    end


    Stimulus = zeros(1, Last)
    X[1, 1] = -0.754;  #Initial conditions here if different from zero
    X[2, 1] = 0.279;  #Initial conditions here if different from zero
    X[3, 1] = -0.754;  #Initial conditions here if different from zero
    X[4, 1] = 0.279;  #Initial conditions here if different from zero
    X[5, 1] = -0.754;  #Initial conditions here if different from zero
    X[6, 1] = 0.279;  #Initial conditions here if different from zero
    X[7, 1] = -0.754;  #Initial conditions here if different from zero
    X[8, 1] = 0.279;  #Initial conditions here if different from zero
    Wt2 = [0 .5 .5 1];  #Second set of RK weights
    rkIndex = [1 1 2 3]
    SorD = 1#input("Stimulate (1) Soma or (2) Apical Dendrite: ")
    Stim = 0.45


    #**********
    gc = 4; #Conductance between compartments
    NaX = 0.05
    #**********

    ST = 10.6
    for T = 2:Last
        for rk = 1:4  #Fourth Order Runge-Kutta
            XH = X[:, T-1] + K[:, rkIndex[rk]]*Wt2[rk]
             Tme =Time[T-1] + Wt2[rk]*DT;  #Time upgrade
            ST = Stim*(Tme .>= 2)*(Tme .<= 25)

        K[1, rk] = DT/Tau*(-(17.81 + 47.58*XH[1] + 33.8*XH[1]^2)*(XH[1] - 0.48) - 26*XH[2]*(XH[1] + 0.95) + gc*(XH[3] - XH[1]) + ST*(SorD == 1));  
        K[2, rk] = DT/TauR*(-XH[2] + 1.29*XH[1] + 0.79 + 3.3*(XH[1] + 0.38)^2)
        K[3, rk] = DT/Tau*(NaX*(-(17.81 + 47.58*XH[3] + 33.8*XH[3]^2)*(XH[3] - 0.48) - 26*XH[4]*(XH[3] + 0.95)) + gc*(XH[1] - XH[3]) + gc*(XH[5] - XH[3]));  
        K[4, rk] = DT/TauR*(-XH[4] + 1.29*XH[3] + 0.79 + 3.3*(XH[3] + 0.38)^2)
        K[5, rk] = DT/Tau*(NaX*(-(17.81 + 47.58*XH[5] + 33.8*XH[5]^2)*(XH[5] - 0.48) - 26*XH[6]*(XH[5] + 0.95)) + gc*(XH[3] - XH[5]) + gc*(XH[7] - XH[5]));  
        K[6, rk] = DT/TauR*(-XH[6] + 1.29*XH[5] + 0.79 + 3.3*(XH[5] + 0.38)^2)
        K[7, rk] = DT/Tau*(NaX*(-(17.81 + 47.58*XH[7] + 33.8*XH[7]^2)*(XH[7] - 0.48) - 26*XH[8]*(XH[7] + 0.95)) + gc*(XH[5] - XH[7]) + ST*(SorD == 2));  
        K[8, rk] = DT/TauR*(-XH[8] + 1.29*XH[7] + 0.79 + 3.3*(XH[7] + 0.38)^2)

     end
        X[:, T] = X[:, T-1] + sum((Weights.*K)', 1)'/6
        Stimulus[T] = ST
    end


    VV = -0.9:0.01:1.5
    DVdt = -0.5*((1.37 + 3.66*VV + 2.6*VV.^2).*(VV - 0.48) - Stim/13)./(VV + 0.95)
    DRdt = 1.29*VV + 0.79 + 3.3*(VV + 0.38).^2

    trace1 = PlotlyJS.scatter(;x=Time, y=100*X[1, :], mode="lines", line_color="red")
    trace2 = PlotlyJS.scatter(;x=Time, y=100*X[7, :], mode="lines", line_color="blue")
    layout = PlotlyJS.Layout(title="Soma Potential (red) and Proximal Dendritic Potential (blue)", xlabel="Time (ms)", ylabel="Potential (mV)")
    p1 = PlotlyJS.plot([trace1, trace2], layout)

    trace1 = PlotlyJS.scatter(;x=VV, y=DVdt, mode="lines", line_color="black")
    trace2 = PlotlyJS.scatter(;x=VV, y=DRdt, mode="lines", line_color="blue")
    trace3 = PlotlyJS.scatter(;x=X[1, :], y=X[2, :], mode="lines", line_color="red")
    layout = PlotlyJS.Layout(title="Phase Space", xaxis_range=[-1, 0.6], yaxis_range=[0, 1])
    p2 = PlotlyJS.plot([trace1, trace2, trace3], layout)

    [p1 p2]
end

ActiveDendrite (generic function with 1 method)

In [2]:
ActiveDendrite()

LoadError: [91mUndefVarError: PlotlyJS not defined[39m

# Dendrite.m

In [64]:
function Dendrite()

    #Postsynaptic potential propagation in a passive dendrite

    X = 0:200; #Length in microns
    V1 = 100
    D = 100; #length constant
    Tau = 10; #time constant

    #**********
    NumPSP = 1;  #Change this for problems
    #**********

    X1 = zeros(1, NumPSP)
    T1 = zeros(1, NumPSP)
    for NN = 1:NumPSP
    X1[NN] = 75#input("Distance of synapse from soma (10, 200) = ")
    if NN > 1; T1[NN] = input("Time of PSP in ms (integer) = "); end
    end
    PSP = 0
    for T = 0.00001:0.25:10
        PSP = V1*exp.(-T/Tau)/(2*D*sqrt.(pi*T/Tau)).*exp.(-Tau*(X - X1[1]).^2/(4*D^2*T))
        if NumPSP > 1
            for S = 2:NumPSP
                PSP = PSP + (T > T1[S])*V1*(exp.(-(T - T1[S])/Tau)/(2*D*sqrt.(pi*(T - T1[S])/Tau))).*exp.(-Tau*(X - X1[S]).^2/(4*D^2*(T - T1[S])))
            end
        end
    end
    
    Plots.plot(X, PSP)
    trace1 = PlotlyJS.scatter(;x=X, y=PSP, mode="lines", line_color="red")
    layout = PlotlyJS.Layout(title="PSP vs. X", xlabel="Distance in Microns", xaxis_range=[0, 200], yaxis_range=[0, V1/50])
    p1 = PlotlyJS.plot([trace1], layout)
    # figure(1), FF = plot(X, PSP); axis([0, 200, 0, V1/50]); set(FF, "LineWidth', 2); xlabel('Distance in microns")

    TT = 0.00001:0.05:10
    PSPtme = zeros(1, length(TT))
    PSPtme = V1*exp.(-TT/Tau)./(2*D*sqrt.(pi*TT/Tau)).*exp.(-Tau*X1[1]^2./(4*D^2*TT))
        if NumPSP > 1
            for S = 2:NumPSP
                PSPtme = PSPtme + V1*(TT > T1[S]).*(exp(-(TT - T1[S])/Tau)./(2*D*sqrt.(pi*(TT - T1[S])/Tau))).*exp(-Tau*X1[S]^2./(4*D^2*(TT - T1[S])))
            end
        end

    trace1 = PlotlyJS.scatter(;x=TT, y=PSPtme, mode="lines", line_color="blue")
    layout = PlotlyJS.Layout(title="PSPtme vs. TT", xlabel="Time (ms)", ylabel="Potential Change at Soma")
    p2 = PlotlyJS.plot([trace1], layout)
    
    [p1 p2]
    # figure(2), F2 = plot(TT, PSPtme, "r-'); xlabel('Time in ms'); ylabel('Potential Change at Soma")
    # set(F2, "LineWidth", 2)
    # PSPmax = maximum(PSPtme)
    # MeanPSP = mean(PSPtme)
end

Dendrite (generic function with 1 method)

In [65]:
Dendrite()

# Diffusion.m
### Must make movie for this

In [89]:

# This might help
# https://discourse.julialang.org/t/on-the-fly-animation-in-ijulia-notebooks/524

# X = -1:0.01:1;
# CC = zeros(1, length(X));

# # figure(1), WW = plot(X, CC, 'r-'); set(WW, 'LineWidth', 3); axis([-1 1 0 6]);
# Film = moviein(30);
# for TT = 1:30;
# 	Tm = TT*0.1 - 0.1;
# 	CC = exp(-0.2*Tm)*(3 + cos(2*pi*X)*exp(-0.1*Tm) + 2*cos(2*5*pi*X)*exp(-2.5*Tm));
# 	WW = plot(X, CC, 'r-'); set(WW, 'LineWidth', 3); axis([-1 1 0 6]);
# 	Film(:, TT) = getframe;
# end;
# movie(Film, 2);


Error handling websocket connection:
InterruptException:Unexpected end of input
 ...when parsing byte with value '0'
Stacktrace:
 [1] _error(::String

LoadError: [91mArgumentError: stream is closed or unusable[39m

# Plateau.m

In [97]:
function Plateau()
    Total_Equations = 4;  #Solve for this number of interacting Neurons
    DT = 0.08;  #Time increment
    Final_Time = 1800;   #Final time value for calculation
    Last = Int64(Final_Time/DT + 1);  #Last time step
    Time = DT*(0:Last-1);  #Time vector
    Tau = 0.97;  #Neural time constants in msec
    TauR = 5.6
    TauC = 2;  #Tau for Ca++ entry
    TauH = 20;  #Tau for Iahp potential
    WTS = [1 2 2 1];  #Runge-Kutta Coefficient weights


        X = Array{Float64}(Total_Equations, Last)
        K = Array{Float64}(Total_Equations, 4)
        Weights = Array{Float64}(Total_Equations, 4)


    for NU = 1:Total_Equations;  #Initialize
        X[NU, :] = zeros(1, Last);  #Vector to store response of Neuron #1
        K[NU, :] = zeros(1, 4);  #Runge-Kutta terms	
        Weights[NU, :] = WTS;  #Make into matrix for efficiency in main loop
    end
    Stimulus = zeros(1, Last)
    X[1, 1] = -0.754;  #Initial conditions here if different from zero
    X[2, 1] = 0.279;  #Initial conditions here if different from zero
    X[3, 1] = -0.754;  #Initial conditions here if different from zero
    Wt2 = [0 .5 .5 1];  #Second set of RK weights
    rkIndex = [1 1 2 3]
    Stim = 1#input("Stimulating current strength (0-2): ")



    #**********
    gc = 0.1; #Conductance between dendrite and soma
    p = 0.37; #Proportion of total cell area in soma
    GAHP = 1
    #**********

    for T = 2:Last
      for rk = 1:4  #Fourth Order Runge-Kutta
        XH = X[:, T-1] + K[:, rkIndex[rk]]*Wt2[rk]
        Tme =Time[T-1] + Wt2[rk]*DT;  #Time upgrade
        ST = Stim*(Tme >= 40)*(Tme < 117) - Stim*(Tme >= 1450)*(Tme < 1525)

        K[1, rk] = DT/Tau*(-(17.81 + 47.58*XH[1] + 33.8*XH[1]^2)*(XH[1] - 0.48) - 26*XH[2]*(XH[1] + 0.95) + gc/p*(XH[3] - XH[1]) + ST);  
        K[2, rk] = DT/TauR*(-XH[2] + 1.29*XH[1] + 0.79 + 3.3*(XH[1] + 0.38)^2)
        K[3, rk] = DT/TauC*(-(XH[3] + 0.754)*(XH[3] + 0.7)*(XH[3] - 1.0) - GAHP*XH[4]*(XH[3] + 0.95) + gc/(1 - p)*(XH[1] - XH[3]))
        K[4, rk] = DT/TauH*(-XH[4] + 0.5*(XH[3] + 0.754))

     end
        X[:, T] = X[:, T-1] + sum((Weights.*K)', 1)'/6
        Stimulus[T] = ST
    end
    
    VV = -0.9:0.01:1
    DVdt = (1/GAHP)*(-(VV + 0.754).*(VV + 0.7).*(VV - 1) + gc/(1-p)*(-VV - 0.754))./(VV + 0.95)
    DRdt = 0.5*(VV + 0.754)
    
    trace1 = PlotlyJS.scatter(;x=Time, y=100*X[1, :], mode="lines", line_color="red")
    trace2 = PlotlyJS.scatter(;x=Time, y=Stimulus*10 - 100, mode="lines", line_color="blue")
    layout = PlotlyJS.Layout(title="Soma Voltage (red) and Dendritic Plateau Potential (blue)", xlabel="Time (ms)", ylabel="Potential (mV)")
    p1 = PlotlyJS.plot([trace1, trace2], layout)
    
    trace1 = PlotlyJS.scatter(;x=VV, y=DVdt, mode="lines", line_color="black")
    trace2 = PlotlyJS.scatter(;x=VV, y=DRdt, mode="lines", line_color="blue")
    trace3 = PlotlyJS.scatter(;x=X[3, :], y=X[4, :], mode="lines", line_color="red")
    layout = PlotlyJS.Layout(title="Dendritic Phase Plane", xlabel="VD", ylabel="C")
    p2 = PlotlyJS.plot([trace1, trace2, trace3], layout)
    
    [p1 p2]
    
#     figure(1), ZA = plot(Time, 100*X[1, :], "r-', Time, Stimulus*10 - 100, 'b-'); set(ZA, 'LineWidth", 2)
#     xlabel("Time (ms)'); ylabel('Potential (mV)")
#     title("Soma Voltage (red) & Dendritic Plateau Potential (blue)")
#     VV = -0.9:0.01:1
#     DVdt = (1/GAHP)*(-(VV + 0.754).*(VV + 0.7).*(VV - 1) + gc/(1-p)*(-VV - 0.754))./(VV + 0.95)
#     DRdt = 0.5*(VV + 0.754)
#     figure(2), ZB = plot(VV, DVdt, "k-', VV, DRdt, 'b-', X[3, :], X[4, :], 'r-"); axis([-1, 1, -0.1, 1])
#     set(ZB, "LineWidth', 2); axis square; title('Dendritic Phase Plane'); xlabel('VD'); ylabel('C")
#     Spikes = (X[1, 1:Last - 1] < -0.12).*(X[1, 2:Last] >= -0.12)
#     SpkTime = zeros(1, sum(Spikes))
#     Nspk = 1;  #Number of spike
#     for T = 1:length(Spikes);  #Calculate spike rate for all interspike intervals
#         if Spikes[T] == 1; SpkTime[Nspk] = T*DT; Nspk = Nspk + 1; end
#     end
#     Final = length(SpkTime)
#     Spike_Rates = 1000./(SpkTime[2:Final] - SpkTime[1:Final - 1])'
end


Plateau (generic function with 1 method)

In [98]:
Plateau()

# Spike_Propagate.m
### Also involves a movie
### Also still need to debug sizes of VL, VR, VV, etc.

In [3]:

Tau = 0.8
TauR = 1.9
DT = 0.01;  #Time increment as fraction of time constant
DX = 0.08; #Delta X spatial increment
X = 0:DX:12
Right = length(X)
Final_Time = 10;   #Final time value for calculation
Last = Int64(Final_Time/DT + 1);  #Last time step
Time = DT*(0:Last-1);  #Time vector
V = -0.704*ones(1, Right);   #Resting potential is -0.704
R = 0.088*ones(1, Right)
Choice = 1#input("Please choose (1) Stimulate left, (2) Stimulate Center, or (3) Stimulate both ends: ")

#**********
ENa = 0.55;  #change for problems
DD = 0.25; #Length constant for diffusion
#**********

#Initial conditions
if Choice == 1 | Choice == 3
    V[1] = -0.065
end  #Excitation at first location
#     if Choice == 3; V[Right] = -0.065; end
#     if Choice == 2; V[round(Right/2)] = -0.065; end

trace1 = PlotlyJS.scatter(;x=X, y=100*V,  mode="lines", line_color="red")
layout = PlotlyJS.Layout(title="Spike Propagation during 10msec", xlabel="Distance along Axon (mm)", ylabel="Membrane Voltage (mV)", xaxis_range=[0, 12], yaxis_range=[-100, 50])
p1 = PlotlyJS.plot([trace1], layout)

#     figure(1), ZA = plot(X, 100*V, "r-'); set(ZA, 'LineWidth", 3)
#     axis([0 12 -100 50])
#     xlabel("Distance along Axon (mm)'); ylabel('Membrane Voltage (mV)")
#     title("Spike Propagation during 10 msec")
#     Pos = 1; #position index for velocity

for T = 2:Last
    global VV
    global V
    global VR
    global VL
    VV = V
    VL = [V[1] V[1:Right - 1]];  #Incorporates zero flux boundary conditions.
    VR = [V[2:Right] V[Right]]
    V = V + DT./Tau.*(-(17.81 + 47.71.*VV + 32.63.*VV.^2).*(VV - ENa) - 26.0.*R.*(VV + 0.92) + (DD.^2/DX.^2).*(VL + VR - 2*VV))
    R = R + DT/TauR*(-R + 1.35*VV + 1.03)
    if rem((T - 2), 40) ==0
        ZA = plot(X, 100*V, "r-'); set(ZA, 'LineWidth", 3); axis([0 12 -100 50])
        xlabel("Distance along Axon (mm)'); ylabel('Membrane Voltage (mV)")
        title("Spike Propagation during 10 msec")
        pause(0.1)
    end
    if (T*DT == 3) | (T*DT == 6); #Calculate positions to determine velocity
        (Mx, J[Pos]) = maximum(V)
        Pos = Pos + 1
    end
end

Spike_Velocity = (X[J[2]] - X[J[1]])/3





LoadError: [91mUndefVarError: PlotlyJS not defined[39m

LoadError: [91mDimensionMismatch("dimensions must match")[39m